<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
   
                                          
                                          
                                          
                                          
                                          
#                                                 Regression with Abalone Data Set: Principal Components Regression




#                                                        Gladys Murage

#                              College of Business, Engineering, and  Technology, National University

#                                         DDS8555 v1: PREDICTIVE ANALYSIS(3602869492)

#                                                        Dr MOHAMED NABEEL

#                                                            March 17, 2025


<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>

# What is Principal Component Regression (PCR)?
Principal Component Regression (PCR) is a technique that combines Principal Component Analysis (PCA) with linear regression. It is useful when you have multicollinearity (prediciting features are correlated) in your data or when you have more features than observations ( high dimensionality). 

## Step 1: Import Necessary Libraries and load the Kaggle data sets

In [9]:
# Import libraries 
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

## Step 2: Prepare the Data
The target variable is in a column named Rings in the train_data

In [10]:
# Separate features and target variable
X = train_data.drop(columns=['Rings'])
y = train_data['Rings']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 3: Find, convert categorical data and handle missing values

In [13]:
# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
if len(categorical_cols) > 0:
    encoder = OneHotEncoder(drop='first')  # Use drop='first' to avoid multicollinearity
    encoded_cols = encoder.fit_transform(X[categorical_cols])
    encoded_cols_df = pd.DataFrame(encoded_cols.toarray(), columns=encoder.get_feature_names_out(categorical_cols))\

# Drop original categorical columns and concatenate encoded columns
    X = X.drop(columns=categorical_cols)
    X = pd.concat([X, encoded_cols_df], axis=1)

# Handle missing values (if any)
X = X.fillna(X.mean())  # Fill missing values with the mean of each column


## Step 4: Split the data for training

In [14]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## Step 5: Standardize the data (Important for PCA)

In [15]:
# Standardize the data (important for PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

## Step 6: Initialize PCA

In [16]:
# Initialize PCA and fit on the training data
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)



## Step 7: Initialize the Linear Regression Model

In [17]:
# Initialize the Linear Regression model
model = LinearRegression()


## Step 8: Fir the model onto the PCA transformed training data

In [18]:
# Fit the model on the PCA-transformed training data
model.fit(X_train_pca, y_train)


LinearRegression()

## Step 9: Predict on the PCA-Transformed Validation data

In [20]:
# Predict on the PCA-transformed validation data
y_val_pred = model.predict(X_val_pca)

## Step 10: Calculate Root Mean Squared Logarithmic Error (RMSLE)

In [22]:
# Calculate RMSLE
def rmsle(y_true, y_pred):
    # Ensure predictions and true values are non-negative
    y_true = np.maximum(0, y_true)
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

rmsle_score = rmsle(y_val, y_val_pred)
print(f"Root Mean Squared Logarithmic Error (RMSLE) on Validation Set: {rmsle_score}")



Root Mean Squared Logarithmic Error (RMSLE) on Validation Set: 0.19474415852230692


## Step 11: Preprocess the test data the same way as the training data

In [87]:
print(X_train.columns)

Index(['id', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1',
       'Whole weight.2', 'Shell weight', 'Sex_I', 'Sex_M'],
      dtype='object')


In [88]:
print(test_data.columns)

Index(['id', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1',
       'Whole weight.2', 'Shell weight', 'Sex_I', 'Sex_M'],
      dtype='object')


In [89]:
from sklearn.preprocessing import OneHotEncoder

# Step 1: Verify column names
print("X_train columns:", X_train.columns)
print("test_data columns:", test_data.columns)

# Step 2: Encode the 'Sex' column in test_data to match X_train
if 'Sex' in test_data.columns:
    # Initialize the OneHotEncoder with the same configuration used for X_train
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

    # Fit the encoder on the unique values of 'Sex' in test_data
    encoder.fit(test_data[['Sex']])

    # Transform the 'Sex' column in test_data
    encoded_test_data = encoder.transform(test_data[['Sex']])
    encoded_column_names = encoder.get_feature_names_out(['Sex'])

    # Add the encoded columns to test_data
    test_data = pd.concat([test_data.drop(columns=['Sex']), 
                           pd.DataFrame(encoded_test_data, columns=encoded_column_names)], axis=1)

    print("test_data after encoding:")
    print(test_data.head())
else:
    print("Column 'Sex' not found in test_data. Skipping encoding.")

X_train columns: Index(['id', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1',
       'Whole weight.2', 'Shell weight', 'Sex_I', 'Sex_M'],
      dtype='object')
test_data columns: Index(['id', 'Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1',
       'Whole weight.2', 'Shell weight', 'Sex_I', 'Sex_M'],
      dtype='object')
Column 'Sex' not found in test_data. Skipping encoding.


## Step 12: Scale the data and apply PCA

In [90]:
# Import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Step 1: Fill missing values in numeric columns with the means
numeric_cols = X_train.select_dtypes(include=['int', 'float']).columns
X_train[numeric_cols] = X_train[numeric_cols].fillna(X_train[numeric_cols].mean())
test_data[numeric_cols] = test_data[numeric_cols].fillna(test_data[numeric_cols].mean())

# Step 2: Standardize numeric columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_cols])
test_data_scaled = scaler.transform(test_data[numeric_cols])

# Step 3: Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(test_data_scaled)

# Display PCA-transformed test data
print("PCA-transformed test data:")
print(X_test_pca)

PCA-transformed test data:
[[ 2.81787426  0.75995272  1.75863694  0.56075429]
 [ 1.76149812  1.07293501  1.77683515  0.2909738 ]
 [ 0.81503066  1.33888992  1.79161837  0.11091342]
 ...
 [ 0.65411004 -1.69066664  3.96499179  0.94741167]
 [ 3.74425065 -1.47592194  3.97260446 -0.34236667]
 [-1.80298474  0.08477942  4.0582052  -1.36234991]]


In [69]:
print("Explained variance ratio:", pca.explained_variance_ratio_)

Explained variance ratio: [0.71702813 0.10382612 0.09994209 0.03428858]


# Breaking down the Principal Component Analysis (PCA) results
## 1. PCA-Transformed Test Data
PCA (Principal Component Analysis) transforms the original features into a new set of uncorrelated features called principal components. Each row in the PCA-transformed test data corresponds to a sample in the test dataset, and each column corresponds to a principal component.

### Example:
For the first sample in the test data:


[2.81787426  0.75995272  1.75863694  0.56075429]

PC1: 2.81787426

PC2: 0.75995272

PC3: 1.75863694

PC4: 0.56075429

These values represent the coordinates of the sample in the new PCA space.

## Key Points:
Each principal component is a linear combination of the original features.

The principal components are ordered by the amount of variance they explain in the data (PC1 explains the most variance, PC2 the second most, and so forth).

The number of principal components is determined by the n_components parameter in PCA. In this case, 4 components were retained.

## 2. Explained Variance Ratio
The explained_variance_ratio_ attribute of PCA tells of the proportion of the dataset’s variance that is explained by each principal component.

### Example:

Explained variance ratio: [0.71702813 0.10382612 0.09994209 0.03428858]

PC1: Explains 71.70% of the variance.

PC2: Explains 10.38% of the variance.

PC3: Explains 9.99% of the variance.

PC4: Explains 3.43% of the variance.

## Key Points:
The sum of the explained variance ratios is 0.954 (71.70% + 10.38% + 9.99% + 3.43%), which means these 4 components explain 95.4% of the total variance in the dataset.

This is consistent with the PCA configuration (n_components=0.95), which retains enough components to explain 95% of the variance.
## 3. What Does This Mean for the PCA  Model?
### Dimensionality Reduction:
PCA reduces the number of features from the original dataset to 4 principal components while retaining 95.4% of the variance.

This simplifies the dataset and can improve model performance by removing noise and redundancy.

### Model Input:
The PCA-transformed data (X_train_pca and X_test_pca) is used as input into the regression model next instead of the original features.

This can help improve model performance, especially if the original dataset had many features or multicollinearity.

### Interpretability:
The principal components are not directly interpretable in terms of the original features, but they capture the most important patterns in the data.

If interpretability is important, one can analyze the PCA loadings (the contribution of each original feature to the principal components).

## 4. How to Use This Information
## Check PCA Loadings (Optional):
In order to  understand how the original features contribute to the principal components,one can examine the PCA loadings:

# Get the PCA loadings (contribution of each original feature to the principal components)
loadings = pca.components_

# Create a DataFrame to visualize the loadings
loadings_df = pd.DataFrame(loadings, columns=X_train.columns, index=[f"PC{i+1}" for i in range(loadings.shape[0])])
print("PCA Loadings:")
print(loadings_df)
Example Output:
PCA Loadings:
         Length  Diameter  Height  Whole weight  Whole weight.1  Whole weight.2  Shell weight  Sex_M
PC1      0.45      0.40    0.12         0.70           0.30           0.11         0.14    0.10
PC2     -0.30     -0.25    0.15        -0.20           0.40          -0.10         0.05   -0.20
PC3      0.10      0.05   -0.20         0.10          -0.10           0.30        -0.15    0.25
PC4     -0.05     -0.10    0.05        -0.05           0.10          -0.05         0.10   -0.15
This shows how each original feature contributes to each principal component.

## 5. Next Steps
Use the PCA-transformed data (X_train_pca and X_test_pca) to train and evaluate the regression model.

If the model performance is satisfactory, make predictions and save the results.

If there is need to improve interpretability, analyze the PCA loadings to understand the contribution of the original features.

# Summary
## PCA-Transformed Test Data: 
The transformed data in the new PCA space, where each column represents a principal component.

## Explained Variance Ratio: 
The proportion of variance explained by each principal component. In your case, 4 components explain 95.4% of the variance.

## Use Case: 
PCA simplifies the dataset and can improve model performance by focusing on the most important patterns in the data.


## Step 13: Using PCA for Regression Modeling:
After transforming the data with PCA, one can use the PCA-transformed features (X_train_pca and X_test_pca) as input to the regression model. 

In [72]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Example: Assume X and y are already loaded
# X: Features, y: Target variable

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Standardize numeric columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Convert scaled data to NumPy arrays
X_train_array = X_train_scaled
X_test_array = X_test_scaled

# Step 4: Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_array)
X_test_pca = pca.transform(X_test_array)

# Step 5: Display PCA-transformed test data
print("PCA-transformed test data:")
print(X_test_pca)

PCA-transformed test data:
[[-1.46827525 -0.72149524 -1.69242492  0.53369047]
 [-2.60735326 -0.74226829  1.09435236  0.37293396]
 [ 0.62934059  1.44702812  0.33091792  0.06617372]
 ...
 [-1.33170141  0.02032609  0.87375738 -1.26454658]
 [ 3.35038075  0.74416895 -0.25922538  0.56534709]
 [ 2.16683811  0.88457398  1.40949824  0.32056082]]


# Explanation of the Code
### Load the Datasets:

Load train.csv, test.csv, and sample_submission.csv into train_data, test_data, and sample_submission, respectively.

### Separate Features and Target:

For the training data, separate features (X_train) and target (y_train).

### Encode Categorical Columns:

If the 'Sex' column exists, encode it using OneHotEncoder for both training and test data.

### Standardize the Data:

Standardize the features using StandardScaler to ensure all features have a mean of 0 and a standard deviation of 1.

### Apply PCA:

Use PCA to reduce the dimensionality of the dataset while retaining 95% of the variance.

### Train the Model:

Train a linear regression model on the PCA-transformed training data.

### Make Predictions:

Use the trained model to make predictions on the PCA-transformed test data.

### Calculate the RMSLE

### Save Predictions:

Save the predictions to the sample_submission.csv file in the correct format.

In [99]:
# Import Libraries
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load the datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Separate features and target variable in the training data
X_train = train_data.drop(columns=["Rings"])  # Replace "Rings" with the actual target column name
y_train = train_data["Rings"]

# Step 1: Encode the 'Sex' column (if it exists)
if 'Sex' in X_train.columns:
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_encoded = encoder.fit_transform(X_train[['Sex']])
    encoded_column_names = encoder.get_feature_names_out(['Sex'])
    X_train = pd.concat([X_train.drop(columns=['Sex']), 
                         pd.DataFrame(X_train_encoded, columns=encoded_column_names)], axis=1)

    # Encode the 'Sex' column in the test data
    X_test_encoded = encoder.transform(test_data[['Sex']])
    test_data = pd.concat([test_data.drop(columns=['Sex']), 
                           pd.DataFrame(X_test_encoded, columns=encoded_column_names)], axis=1)

# Step 2: Standardize the data (important for PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_data)

# Step 3: Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Step 4: Train a model on PCA-transformed training data
model = LinearRegression()
model.fit(X_train_pca, y_train)

# Step 5: Make predictions on PCA-transformed test data
y_pred = model.predict(X_test_pca)

# Step 6: Calculate RMSLE on the validation set
rmsle_score = rmsle(y_val, y_val_pred)
print("RMSLE on validation set:", rmsle_score)

# Step 7: Save predictions to the sample_submission file
sample_submission["Rings"] = y_pred  # Assuming 'Rings' is the target column
sample_submission.to_csv("submission5.csv", index=False)
print("Predictions saved to submission5.csv")

RMSLE on validation set: 0.20025094970075333
Predictions saved to submission5.csv


# Breaking down the results of the RMSLE of the PCA on a Linear regression model
The RMSLE (Root Mean Squared Logarithmic Error) value of 0.20025094970075333 on the validation set provides insight into the performance of the PCA-based linear regression model. 
## 1. What is RMSLE?
RMSLE is a metric used to evaluate the accuracy of regression models. It measures the difference between the logarithm of the predicted values and the logarithm of the true values.

### Key Properties of RMSLE:
Penalizes Underestimation More Than Overestimation:
RMSLE is asymmetric and penalizes underestimates more heavily than overestimates.

### Logarithmic Scale:
It works on a logarithmic scale, making it suitable for targets with a wide range of values (e.g., house prices, sales, etc.).

### Non-Negative Values:
Both true and predicted values must be non-negative.

## 2. What Does RMSLE = 0.200 Mean?
An RMSLE of 0.200 indicates the average logarithmic error between the predicted and true values. The interpretaation is aas follows: 

Lower RMSLE is Better:

An RMSLE of 0 means perfect predictions (no error).

An RMSLE of 0.200 suggests that, on average, the model’s predictions are off by approximately 0.200 on the logarithmic scale.

### Relative Interpretation:

If the true target values are in the range of 1 to 100, an RMSLE of 0.200 indicates that the model’s predictions are reasonably accurate but still have room for improvement.

If the true target values are in the range of 1 to 10, an RMSLE of 0.200 suggests that the model’s predictions are less accurate.

## 3. How Does PCA Affect the Model’s Performance?
PCA (Principal Component Analysis) is used for dimensionality reduction. It transforms the original features into a smaller set of uncorrelated features (principal components) that capture the most variance in the data.

### Impact of PCA on RMSLE:
#### Improved Performance:

If the original dataset has many irrelevant or redundant features, PCA can improve model performance by focusing on the most important patterns in the data.

This can lead to a lower RMSLE.

#### Potential Loss of Information:

If PCA removes too much variance (e.g., by retaining too few components), the model may lose important information, leading to a higher RMSLE.

#### Reduced Overfitting:

PCA can help reduce overfitting by removing noise and redundancy in the data, which can improve generalization to unseen data.

## 4. What Does RMSLE = 0.200 Mean for Your Model?
### Model Performance:

The model’s predictions are reasonably accurate but could be improved.

An RMSLE of 0.200 suggests that the model is capturing some of the underlying patterns in the data but may still be missing some important relationships.

### PCA’s Role:

The PCA transformation likely helped reduce the dimensionality of the dataset and improved the model’s performance by focusing on the most important features.

However, the RMSLE value indicates that there is still room for improvement, either by tuning the PCA parameters (e.g., increasing the number of components) or by using a more sophisticated model.

## 5. How to Improve the Model’s Performance
If trying to to reduce the RMSLE further, consider the following steps:

### a) Tune PCA Parameters:
Increase the number of principal components retained (e.g., n_components=0.99 to retain 99% of the variance).

Experiment with different values of n_components to find the optimal balance between dimensionality reduction and information retention.

### b) Feature Engineering:
Create new features or transform existing ones to better capture the relationships in the data.

For example, if the target variable has a logarithmic distribution, apply a logarithmic transformation to the target before training the model.

### c) Try a Different Model:
Linear regression may not be the best model for the data. Consider using more advanced models like:

Random Forest Regressor

Gradient Boosting Regressor (e.g., XGBoost, LightGBM, or CatBoost)

Support Vector Regressor (SVR)

### d) Hyperparameter Tuning:
Use techniques like Grid Search or Randomized Search to find the best hyperparameters for the model.

### e) Cross-Validation:
Use cross-validation to get a more robust estimate of the model’s performance and reduce overfitting.

## 6. Next Steps
### Analyze Residuals:

Plot the residuals (true values vs. predicted values) to identify patterns in the errors. This helps in understanding where the model is underperforming.

### Feature Importance:

If using a model that supports feature importance (e.g., Random Forest), analyze which features are most important for making predictions.

### Experiment with Different Models:

Try more advanced models and compare their RMSLE values to see if  better performance can be achieved.